# HeadySoul GPU Node — ML-Powered Mission Alignment Scoring

```
╔══════════════════════════════════════════════════════════════════╗
║  ∞ SACRED GEOMETRY ∞  Organic Systems · Breathing Interfaces    ║
║  NODE: HeadySoul Colab Pro+ GPU                                 ║
║  PURPOSE: Semantic similarity scoring against mission values     ║
╚══════════════════════════════════════════════════════════════════╝
```

**What this does:**
- Uses sentence-transformers on GPU to compute semantic similarity between tasks and mission values
- Exposes a FastAPI server via ngrok tunnel
- Auto-registers with HeadyCloud at headysystems.com
- Sends heartbeats every 30s to stay in the cluster

**Branded domains only:** headysystems.com | headycloud.com | headyconnection.com

In [ ]:
# Cell 1: Install dependencies + verify GPU
!pip install -q sentence-transformers fastapi uvicorn pyngrok pyyaml httpx

import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB')
else:
    print('WARNING: No GPU detected — running on CPU (slower)')

In [ ]:
# Cell 2: Configuration — Branded domains (now proxying to Render)
import os

HEADY_CONFIG = {
    'node_id': 'heady-soul-gpu',
    'node_role': 'soul-scorer',
    'port': 5000,
    'cloud_layers': {
        'headysystems': 'https://headyio.com',
        'headyme': 'https://headycloud.com',
        'headyconnection': 'https://headyconnection.com',
    },
    'registration_endpoints': [
        'https://headyio.com/api/soul/colab/connect',
        'https://headycloud.com/api/soul/colab/connect',
        'https://headyconnection.com/api/soul/colab/connect',
    ],
    'api_key': os.environ.get('HEADY_API_KEY', ''),
}

# Mission value weights (must match heady-soul.yaml)
VALUE_WEIGHTS = {
    'access': 0.30,
    'fairness': 0.25,
    'intelligence': 0.20,
    'happiness': 0.15,
    'redistribution': 0.10,
}

THRESHOLDS = {
    'veto': 40,
    'escalate': 60,
    'auto_approve': 75,
}

print('HeadySoul GPU Node configured')
print(f'Values: {VALUE_WEIGHTS}')

In [ ]:
# Cell 3: Load sentence-transformer model on GPU
from sentence_transformers import SentenceTransformer
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
print(f'Model loaded on {device}')

# Pre-compute mission dimension embeddings
MISSION_TEXTS = {
    'access': 'expand access to underserved users free nonprofit students PPP pricing purchasing power parity inclusive education community low-income global south multilingual offline mobile-first open-source',
    'fairness': 'no lock-in no extraction co-ownership open transparent ethical consent privacy fair equitable democratic honest trust accountability no dark patterns portable exportable',
    'intelligence': 'AI learning self-improving adaptive intelligent optimization pattern recognition prediction analysis insight autonomous self-correcting evolution neural network machine learning',
    'happiness': 'user joy delight satisfaction UX user experience beautiful intuitive simple fast responsive pleasant friendly helpful supportive empowering wellbeing positive',
    'redistribution': 'wealth redistribution revenue sharing donation profit sharing cooperative mutual aid social impact giveback scholarship subsidy pay-what-you-can sliding scale fair compensation',
}

# Sacred Geometry alignment text
SACRED_GEOMETRY_TEXT = 'organic breathing deterministic self-correcting fractal natural rhythmic renewal healing coherent harmonic sacred geometry golden ratio fibonacci'

MISSION_VECTORS = {dim: model.encode(text) for dim, text in MISSION_TEXTS.items()}
SG_VECTOR = model.encode(SACRED_GEOMETRY_TEXT)

print(f'Mission embeddings computed ({len(MISSION_VECTORS)} dimensions + Sacred Geometry)')

In [ ]:
# Cell 4: Scoring functions
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import json

def score_task(task):
    """Score task alignment with mission values using GPU-accelerated semantic similarity"""
    task_text = f"{task.get('title', '')} {task.get('description', '')} {task.get('type', '')} {json.dumps(task.get('metadata', {}))}"
    task_vec = model.encode(task_text)

    breakdown = {}
    for dim, mission_vec in MISSION_VECTORS.items():
        sim = float(cosine_similarity([task_vec], [mission_vec])[0][0])
        breakdown[dim] = round(max(0, min(100, sim * 100)), 2)

    # Sacred Geometry bonus
    sg_sim = float(cosine_similarity([task_vec], [SG_VECTOR])[0][0])
    sacred_geometry = round(max(0, min(1, sg_sim)), 4)

    # Weighted total
    total = sum(breakdown[dim] * VALUE_WEIGHTS[dim] for dim in breakdown)
    total += sacred_geometry * 0.15 * 100  # Sacred Geometry bonus (up to 15 points)
    total = round(min(100, total), 2)

    return {
        'total': total,
        'breakdown': breakdown,
        'sacred_geometry': sacred_geometry,
        'veto': total < THRESHOLDS['veto'],
        'escalate': total >= THRESHOLDS['veto'] and total < THRESHOLDS['escalate'],
        'auto_approve': total >= THRESHOLDS['auto_approve'],
        'scored_by': 'colab_gpu',
        'model': 'all-MiniLM-L6-v2',
        'device': device,
        'timestamp': datetime.now().isoformat(),
    }

def score_strategies(strategies, context=''):
    """Batch score strategies for Monte Carlo filtering"""
    results = []
    for s in strategies:
        desc = f"{s.get('name', '')} {json.dumps(s.get('steps', []))} {context}"
        score = score_task({'description': desc, 'type': 'strategy'})
        results.append({'strategy_id': s.get('id'), 'alignment': score['total'], 'breakdown': score['breakdown']})
    return results

# Quick test
test_results = [
    ('PPP pricing for nonprofits', score_task({'title': 'Add PPP pricing for nonprofits', 'description': 'Purchasing power parity tiers for underserved communities', 'metadata': {'targetUsers': ['free', 'nonprofit']}})),
    ('Dark pattern engagement hack', score_task({'title': 'Add viral growth hack', 'description': 'Dark pattern to increase engagement and lock users in'})),
    ('Self-improving AI pipeline', score_task({'title': 'ML pipeline optimization', 'description': 'Self-improving adaptive learning system for better predictions'})),
]

print('=== HeadySoul GPU Scoring Test ===')
for name, result in test_results:
    status = 'VETO' if result['veto'] else 'ESCALATE' if result['escalate'] else 'APPROVED'
    print(f'  {name}: {result["total"]:.1f}/100 [{status}]')

In [ ]:
# Cell 5: FastAPI server
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
import uvicorn
import threading
import asyncio
import httpx
import time

app = FastAPI(title='HeadySoul GPU Node', version='1.0.0')

@app.get('/health')
async def health():
    return {
        'status': 'active',
        'node_id': HEADY_CONFIG['node_id'],
        'node_role': HEADY_CONFIG['node_role'],
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
        'model': 'all-MiniLM-L6-v2',
        'dimensions': list(VALUE_WEIGHTS.keys()),
    }

@app.post('/api/soul/evaluate')
async def evaluate_task(request: Request):
    task = await request.json()
    result = score_task(task)
    return result

@app.post('/api/soul/filter-strategies')
async def filter_strategies(request: Request):
    data = await request.json()
    strategies = data.get('strategies', [])
    context = data.get('context', '')
    min_score = data.get('minAlignmentScore', THRESHOLDS['escalate'])
    scored = score_strategies(strategies, context)
    approved = [s for s in scored if s['alignment'] >= min_score]
    rejected = [s for s in scored if s['alignment'] < min_score]
    return {'approved': approved, 'rejected': rejected, 'total': len(strategies), 'passed': len(approved)}

@app.get('/api/soul/values')
async def get_values():
    return {'weights': VALUE_WEIGHTS, 'thresholds': THRESHOLDS}

print('FastAPI app created with endpoints: /health, /api/soul/evaluate, /api/soul/filter-strategies, /api/soul/values')

In [ ]:
# Cell 6: ngrok tunnel + auto-register with HeadyCloud
from pyngrok import ngrok, conf

# Authenticate ngrok
conf.get_default().auth_token = "39ZBirdUD63xgta7yN7OFZpE84m_3QZyJTDno1b8Yhv9Nfy8s"

# Start ngrok tunnel
public_url = ngrok.connect(HEADY_CONFIG['port']).public_url
print(f'HeadySoul GPU Node live at: {public_url}')

PUBLIC_URL = public_url

async def register_with_clouds():
    """Register this Colab GPU node with all HeadyCloud layers"""
    async with httpx.AsyncClient(timeout=15) as client:
        for endpoint in HEADY_CONFIG['registration_endpoints']:
            try:
                resp = await client.post(endpoint, json={'url': PUBLIC_URL})
                if resp.status_code == 200:
                    print(f'  Registered with {endpoint}: {resp.json()}')
                else:
                    print(f'  Registration pending: {endpoint} ({resp.status_code})')
            except Exception as e:
                print(f'  Could not reach {endpoint}: {e}')

async def heartbeat_loop():
    """Send heartbeats every 30s to all cloud layers"""
    while True:
        await asyncio.sleep(30)
        async with httpx.AsyncClient(timeout=10) as client:
            for layer, url in HEADY_CONFIG['cloud_layers'].items():
                try:
                    await client.post(f'{url}/api/nodes/heartbeat', json={
                        'node_id': HEADY_CONFIG['node_id'],
                        'status': 'active',
                        'url': PUBLIC_URL,
                        'metrics': {
                            'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
                            'gpu_memory_used': torch.cuda.memory_allocated(0) / 1e9 if torch.cuda.is_available() else 0,
                            'model': 'all-MiniLM-L6-v2',
                        }
                    })
                except:
                    pass

# Register and start heartbeats
asyncio.run(register_with_clouds())
threading.Thread(target=lambda: asyncio.run(heartbeat_loop()), daemon=True).start()
print(f'HeadySoul GPU node live at {PUBLIC_URL} — accepting scoring requests from HeadyCloud')
# Colab VM bind address for uvicorn (dual-stack all-interfaces)
uvicorn.run(app, host='::', port=HEADY_CONFIG['port'])